<a href="https://colab.research.google.com/github/ekgren/workshop/blob/main/Day2/QA_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install datasets
!pip install tokenizers

In [ ]:
import datasets
import transformers
import torch
import copy
import tqdm

In [ ]:
tokenizer = transformers.AutoTokenizer.from_pretrained('KB/bert-base-swedish-cased')
kb_bert = transformers.AutoModel.from_pretrained('KB/bert-base-swedish-cased')

In [ ]:
dataset = datasets.load_dataset('stsb_mt_sv')
train_ds = dataset['train']
test_ds = dataset['test']
eval_ds = dataset['validation']


In [ ]:
def encode(*texts):
  assert 1 <= len(texts) <= 2
  return tokenizer(*texts, padding=True, truncation=True, max_length=512, return_tensors='pt')

In [ ]:
train_ds[0]

In [ ]:
def collate_paired(rows):
  s1s = [row['sentence1'] for row in rows]
  s2s = [row['sentence2'] for row in rows]
  scores = torch.tensor([row['score'] for row in rows])
  return encode(s1s, s2s), scores

def collate_dual(rows):
  s1s = [row['sentence1'] for row in rows]
  s2s = [row['sentence2'] for row in rows]
  scores = torch.tensor([row['score'] for row in rows])
  return encode(s1s), encode(s2s), scores


In [ ]:
COLLATER = collate_paired
train_dl = torch.utils.data.DataLoader(  
    train_ds,
    collate_fn=COLLATER,
    shuffle=True,
    batch_size=8,
    pin_memory=True,
  )
test_dl = torch.utils.data.DataLoader(  
    test_ds,
    collate_fn=COLLATER,
    shuffle=False,
    batch_size=8,
    pin_memory=True,
  )
eval_dl = torch.utils.data.DataLoader(  
    eval_ds,
    collate_fn=COLLATER,
    shuffle=False,
    batch_size=8,
    pin_memory=True,
  )

In [ ]:

class PairedModel(torch.nn.Module):
  def __init__(self):
    super().__init__()
    self.model = copy.deepcopy(kb_bert)
    self.linear = torch.nn.Linear(768, 1)
  
  def forward(self, data):
    return self.linear(self.model(**data)['pooler_output']).squeeze(-1)

In [ ]:

batch, score = next(iter(train_dl))

In [ ]:

paired_model(batch)

In [ ]:
paired_model = PairedModel().cuda()
optimizer = torch.optim.AdamW(paired_model.parameters())
EPOCHS = 1
for epoch in range(EPOCHS):
  batches = tqdm.tqdm(train_dl)
  for batch, score in batches:
    batch = {k : v.cuda() for k, v in batch.items()}
    score = score.cuda()
    optimizer.zero_grad()

    prediction = paired_model(batch)
    
    loss = torch.nn.functional.mse_loss(prediction, score)

    loss.backward()

    optimizer.step()

    batches.set_description('{:.2f}'.format(loss.item()))


# Exempel KB-Bert med ordmaskning

När man förtränar Bert modeller så lär de sig språklig statistik genom att se massa text, maska ord och gissa vilket ord som bör vara på en den maskerade platsen. I slutändan är det nästan aldrig gissa ord som modellerna används till utan man anpassar (finetunear) dem till en annan uppgift. Men för att illustrera hur den förtränade modellen fungerar så gör vi en maskningsuppgift.

Vi börjar med att ladda KB-Bert och dess tokeniserare.

In [ ]:
tokenizer = transformers.AutoTokenizer.from_pretrained('KB/bert-base-swedish-cased')
model = transformers.BertForMaskedLM.from_pretrained('KB/bert-base-swedish-cased')
print(type(model))

Vi hittar på en exempelmening.

In [ ]:
example = 'Hej och välkommen till Trafikverket! Myndigheten för dig som gillar vägar, bilar och tåg.'
example

Bert är tränad med speciella ord i början och slutet av meningar, [CLS] och [SEP]. Modellen förutsätter att de är med när du stoppar in en mening. Om du skapar en batch med en huggingface tokeniserare görs detta automatiskt av tokeniseraren men i det här exemplet lägger vi till dem manuellt till exempelmeningen. 

In [ ]:
example_preprocessed = f'[CLS] {example} [SEP]'
example_preprocessed

Nu har vi vår mening i textform och nästa steg är att dela upp den i tokens med vår tokeniserare.

In [ ]:
tokens = tokenizer.tokenize(example_preprocessed)
print(tokens)

Sedan gör vi om våra tokens till index som modellen använder för att ta fram en sifferrepresentation av de tokens som går in i modellen.

In [ ]:
indexed_tokens = tokenizer.convert_tokens_to_ids(tokens)
print(indexed_tokens)

Nu till själva uppgiften som vi skall utföra med modellen. Vi väljer ut ett ord som ligger på plats 5 med nollindexering, "Trafikverket", och ersätter det med en [MASK] token. Detta för att sedan låta modellen givet resten av meningen gissa vilket ord som passar bäst in istället för [MASK].

In [ ]:
masked_index = 5
tokens[masked_index] = '[MASK]'
print(tokens)
indexed_tokens = tokenizer.convert_tokens_to_ids(tokens)
print("\nToken index:", indexed_tokens)

Sedan matar vi in vår exempelmening i KB-Bert

In [ ]:
_ = model.eval() # Sätter modellen i evalueringsläge för att spara minne. Då räknar den inte fram några gradienter.

with torch.no_grad():
    outputs = model(torch.tensor([indexed_tokens]))

predictions = outputs[0]
print(predictions.shape)

In [ ]:
predicted_index = torch.topk(predictions[0, masked_index], k=5)
print(predicted_index.indices)

In [ ]:
predicted_token = tokenizer.convert_ids_to_tokens(predicted_index.indices)
print(predicted_token)

# Namnigenkänning (Named Entity Recognition)

Exempel: Namnigenkänning

Kort förklaring av namnigenkänning

BERT base fine-tuned for Swedish NER. This model is fine-tuned on the SUC 3.0 dataset.

Entity types used are TME for time, PRS for personal names, LOC for locations, EVN for events and ORG for organisations.

In [ ]:
from transformers import pipeline

nlp = pipeline('ner', model='KB/bert-base-swedish-cased-ner', tokenizer='KB/bert-base-swedish-cased-ner')

In [ ]:
nlp('Hej jag heter Arne och jag vill byta lösenord.')